In [17]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table, ctx
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "123abc"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))


# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)
image_path = '/assets/grazioImage.png'

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Grazio Salvare'))),
    
    html.Img(src=image_path, style={'height': '10%','width': '10%'}),
    html.Hr(),
    html.H1("Unique Handle: Charles Breuer CS-340 T5531"),
    html.Div(className='buttonRow',
             style={'display': 'flex'},
             children=[
                 html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                 html.Button(id='submit-button-two', n_clicks=0, children='Mountain Rescue'),
                 html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue'),
                 html.Button(id='submit-button-four', n_clicks=0, children='Reset')
                 
                     
            ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        selected_rows=[0],
        column_selectable="single", 
        row_selectable="single", 
        page_size= 10,

    ),
    html.Br(),
    html.Hr(),
    html.H1("Unique Handle: Charles Breuer CS-340 T5531"),
    html.H2("Count of Dogs for each type of Breed"),
    
    html.Div( style={'margin': 'auto'}, children=[
        html.Div(dcc.Graph(figure={}, id = 'graph-id'))
    ]),
     html.H2("Location of Currently Selected Animal"),
    html.Div(style={'margin': 'auto'}, children=[
        html.Div(style={'margin': 'auto', 'display': 'inline-block'}, id='map-id')
        
    ])
    
    
], style={'textAlign': 'center'})

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#F2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('datatable-id', 'data'),
    [Input('submit-button-one', 'n_clicks'),
    Input('submit-button-two', 'n_clicks'),
    Input('submit-button-three', 'n_clicks'),
    Input('submit-button-four', 'n_clicks')])
def on_click_table(button1, button2, button3, button4):
    
    if (int(button1) > 0 and ctx.triggered_id == 'submit-button-one' ):
        df = pd.DataFrame.from_records(shelter.read({"animal_type" : "Dog", 
                                                     "sex_upon_outcome": "Intact Female",
                                                     "breed": { "$in": ["Labrador Retriever Mix",
                                                                        "Chesapeake Bay Retriever",
                                                                        "Newfoundland"] },
                                                     "age_upon_outcome_in_weeks": {"$gt": 26.0, "$lt": 156.0}
                                                    }))
    elif (int(button2) > 0 and ctx.triggered_id == 'submit-button-two' ):
        df = pd.DataFrame.from_records(shelter.read({"animal_type" : "Dog",
                                                    "sex_upon_outcome": "Intact Male",
                                                    "breed": {"$in": ["German Shepherd",
                                                                      "Alaskan Malamute",
                                                                      "Old English Sheepdog",
                                                                      "Siberian Husky",
                                                                      "Rottweiler"]},
                                                     "age_upon_outcome_in_weeks": {"$gt": 26.0, "$lt": 156.0}
                                                    }))
    elif (int(button3) > 0 and ctx.triggered_id == 'submit-button-three' ):
        df = pd.DataFrame.from_records(shelter.read({"animal_type" : "Dog",
                                                    "sex_upon_outcome": "Intact Male",
                                                    "breed": {"$in": ["Doberman Pinscher",
                                                                      "German Shepherd",
                                                                      "Golden Retriever",
                                                                      "Bloodhound",
                                                                      "Rottweiler"]},
                                                    "age_upon_outcome_in_weeks": {"$gt": 20.0, "$lt": 300.0}
                                                    }))
    elif (int(button4) > 0 and ctx.triggered_id == 'submit-button-four' ):
        df = pd.DataFrame.from_records(shelter.read({}))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')

@app.callback(
    Output(component_id='graph-id', component_property="figure"),
    [Input('submit-button-one', 'n_clicks'),
    Input('submit-button-two', 'n_clicks'),
    Input('submit-button-three', 'n_clicks'),
    Input('submit-button-four', 'n_clicks')])
def on_click_graph(button1, button2, button3, button4):
    
    if (int(button1) > 0 and ctx.triggered_id == 'submit-button-one' ):
        dff = shelter.read({"animal_type" : "Dog", 
                                                     "sex_upon_outcome": "Intact Female",
                                                     "breed": { "$in": ["Labrador Retriever Mix",
                                                                        "Chesapeake Bay Retriever",
                                                                        "Newfoundland"] },
                                                     "age_upon_outcome_in_weeks": {"$gt": 26.0, "$lt": 156.0}
                                                    },
                                                    {"breed": 1}
                                                   )
    elif (int(button2) > 0 and ctx.triggered_id == 'submit-button-two' ):
        dff = shelter.read({"animal_type" : "Dog",
                                                    "sex_upon_outcome": "Intact Male",
                                                    "breed": {"$in": ["German Shepherd",
                                                                      "Alaskan Malamute",
                                                                      "Old English Sheepdog",
                                                                      "Siberian Husky",
                                                                      "Rottweiler"]},
                                                     "age_upon_outcome_in_weeks": {"$gt": 26.0, "$lt": 156.0}
                                                    },
                                                   {"breed": 1}
                                                   )
    elif (int(button3) > 0 and ctx.triggered_id == 'submit-button-three' ):
        dff = shelter.read({"animal_type" : "Dog",
                                                    "sex_upon_outcome": "Intact Male",
                                                    "breed": {"$in": ["Doberman Pinscher",
                                                                      "German Shepherd",
                                                                      "Golden Retriever",
                                                                      "Bloodhound",
                                                                      "Rottweiler"]},
                                                    "age_upon_outcome_in_weeks": {"$gt": 20.0, "$lt": 300.0}
                                                    },
                                                   {"breed": 1}
                                                   )
    elif (int(button4) > 0 and ctx.triggered_id == 'submit-button-four' ):
        dff = shelter.read({}, {"breed": 1})
    else:
        dff = shelter.read({}, {"breed": 1})
        
    allBreedsWithCount = []
    for breed in dff:
        if "breed" in breed.keys():
            breedFound = False
            for breedCount in allBreedsWithCount:
                if (breed["breed"] == breedCount["name"]):
                    breedCount["count"]+= 1
                    breedFound = True
                    break
            if (breedFound == False):
                newBreed = {
                    "name": breed["breed"],
                    "count": 1
                }
                allBreedsWithCount.append(newBreed)
    allBreedsWithCount = sorted(allBreedsWithCount, key=lambda x: x['name'])
    xValue = []
    yValue = []
    for breedCount in allBreedsWithCount:
        xValue.append(breedCount["name"])
        yValue.append(breedCount["count"])
        
    return px.bar(df, x=xValue, y=yValue, labels={'x': 'Breed', 'y':'Count'})


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
    animalName = ''
    if (dff.iloc[row,9] is None or dff.iloc[row,9] == ''):
        animalName = 'No Name'
    else:
        animalName = dff.iloc[row,9]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
        center=[30.75,-97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        # Marker with tool tip and popup
        # Column 13 and 14 define the grid-coordinates for 
        # the map
        # Column 4 defines the breed for the animal
        # Column 9 defines the name of the animal
        dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
            children=[
            dl.Tooltip(dff.iloc[row,4]),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(animalName)
             ])
          ])
       ])
    ]

    

app.run_server(debug=True)

Dash app running on http://127.0.0.1:27722/
